# Update profiles document of the IUCN Global Ecosystem typology

Scripts by José R. Ferrer-Paris

This script is used to:

- Update format of `docx` document to add/edit details on maps

## Set-up
Load libraries we will need

In [1]:
from docx import Document
import os
from pathlib import Path
import re

## Path for input/output of documents
The original document is saved in a temporal directory:

In [2]:
inputdir = Path(os.path.expanduser('~')) / "tmp" / "typology-web-update-content"
outputdir = inputdir
#os.listdir(inputdir)
filename='Keith_etal_EarthsEcosystems_Appendix S4_3rdrevisions_subm+DK.docx'

## Add caveat about maps
As per reviewer request, we need to add caveats to the maps. Easiest way is to apply same message to all maps

In [3]:
map_msg = 'Thumbnail map for illustrative purposes only. Users should consult sources and Table S4.1 to ensure data quality meets needs of their application.'

### Option one, add message to footer of each section 
We assume that all sections with 'Contributors' in the header is a profile with a thumbnail map, and thus requires a caveat message in the footer. 

In [4]:
profiles = Document(inputdir/filename)
from docx.shared import Pt
for section in profiles.sections:
    h = section.header
    f = section.footer
    #print(section)
    f.is_linked_to_previous = False
            
    for p in h.paragraphs:
        if re.search("^Contributors",p.text):
            #print(p.text)
            run = f.add_paragraph().add_run()
            font = run.font
            font.size = Pt(8)
            run.text = map_msg
        
profiles.save(outputdir / 'demo-footnote-no-pages.docx')

The resulting file does not have page numbers in the profiles.

### Option two, add message to the end of the 'Distribution' paragraph 


In [7]:
profiles = Document(inputdir/filename)
for k in range(19,len(profiles.paragraphs)):
    p=profiles.paragraphs[k]
    if re.search("^Distribution:",p.text):
        run = p.add_run()
        font = run.font
        font.size = Pt(8)
        run.text = " " + map_msg
profiles.save(outputdir / 'demo-append-distribution-section.docx')

### Option three, add message + page number to footer of each section 

Solution to add page number from https://stackoverflow.com/questions/56658872/add-page-number-using-python-docx

First define these functions:

In [8]:
from docx.oxml import OxmlElement, ns
from docx.enum.text import WD_ALIGN_PARAGRAPH
def create_element(name):
    return OxmlElement(name)

def create_attribute(element, name, value):
    element.set(ns.qn(name), value)


def add_page_number(run):
    fldChar1 = create_element('w:fldChar')
    create_attribute(fldChar1, 'w:fldCharType', 'begin')

    instrText = create_element('w:instrText')
    create_attribute(instrText, 'xml:space', 'preserve')
    instrText.text = "PAGE"

    fldChar2 = create_element('w:fldChar')
    create_attribute(fldChar2, 'w:fldCharType', 'end')

    run._r.append(fldChar1)
    run._r.append(instrText)
    run._r.append(fldChar2)

Now we can run:

In [11]:
profiles = Document(inputdir / filename)
for section in profiles.sections:
    h = section.header
    f = section.footer   
    #print(section)
    f.is_linked_to_previous = False
            
    for p in h.paragraphs:
        if re.search("^Contributors",p.text):
            #print(p.text)
            f.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
            pagerun=f.paragraphs[0].add_run()
            add_page_number(pagerun)
            run = f.add_paragraph().add_run()
            font = run.font
            font.size = Pt(8)
            run.text = map_msg
            
profiles.save(outputdir / 'demo-footnote-with-page-numbers.docx')


# That's it!